# CNN-RNN

In [1]:
import tensorflow.keras as keras

In [2]:
# Módulos necesarios. Asegurarse de poder importarlos.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from pylab import rcParams
from scipy import stats
from sklearn import metrics
import seaborn as sns
from sklearn import preprocessing
import os as os
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
from time import time
from tensorflow.keras.models import load_model

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

In [3]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
tf.__version__

/usr/bin/python
3.6.8 (default, Aug 20 2019, 17:12:48) 
[GCC 8.3.0]
sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)


'1.14.0'

In [4]:
keras.__version__

'2.2.4-tf'

# 12-FOLD

In [5]:
subjects = ['01', '02', '03', '05', '08', '09', '10', '11', '13', '14', '16', '17']
global activities
activities = [9,10,11,12,13,19,20,21,24,25,31]

In [6]:
def perf_measure(cm1):
    temp = 0
    TruePositive = np.diag(cm1)
    FalsePositive = []
    for i in range(len(activities)):
        FalsePositive.append(sum(cm1[:,i]) - cm1[i,i])
    FalseNegative = []
    for i in range(len(activities)):
        FalseNegative.append(sum(cm1[i,:]) - cm1[i,i])
    TrueNegative = []
    for i in range(len(activities)):
        temp = np.delete(cm1, i, 0)   # delete ith row
        temp = np.delete(temp, i, 1)  # delete ith column
        TrueNegative.append(sum(sum(temp)))
    
    return(TruePositive, FalsePositive, TrueNegative, FalseNegative)

In [7]:
# Ajustado a los datos
n_time_steps = 128
n_classes = 11 # Nº de clases (el 0 está eliminado)
n_channels = 1 # Nº de canales
n_columns = 40

# Podríamos variarlo
batch_size = 128 # Tamaño del batch
learning_rate = 1e-3 # Learning rate (por defecto es 0.001)
epochs = 100 # Épocas

In [8]:
accuracies = np.zeros((12))
#accuraciesClass = np.zeros((12,11))
precision = np.zeros((12,11))
recall = np.zeros((12,11))
fScore = np.zeros((12,11))
support = np.zeros((12,11))
specifities = np.zeros((12,11))

classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
measure = ['precision', 'recall', 'f1-score', 'support']

for i in range(12):
    print('##################################################################################################################################')
    print('#####################################################  K-FOLD %d  #################################################################'%(i+1))
    
    subject_test = subjects[i]
    print('Test subject: ' + str(subject_test))
    
    ##  GENERACIÓN DATASET
    ts_seg = np.load('./augment_norm/subject_'+subject_test+'_seg.npy')
    ts_lab = np.load('./augment_norm/subject_'+subject_test+'_lab.npy')
    
    limit = 40
    ts_seg = ts_seg[:,:,:,:limit]
    ####################################################################################################################
    ## SHUFFLE DE DATOS
    np.random.seed(235)
    ts_seg = np.reshape(ts_seg[np.random.shuffle(np.arange(0,ts_seg.shape[0]))], (2375,1,128,limit))
    ts_lab = np.reshape(ts_lab[np.random.shuffle(np.arange(0,ts_seg.shape[0]))], (2375,11))
    
    print('Test dataset: ')
    print(ts_seg.shape, ts_lab.shape)
    ####################################################################################################################
    ## RED
    model = load_model('./01AUGNORMFFT_SP_KFOLD_models/CNN_'+subject_test+'.h5')
    model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=learning_rate),metrics=['categorical_accuracy'])
    ####################################################################################################################
    ## TEST
    predictions = model.predict(ts_seg, batch_size = batch_size, verbose = 0)
    
    y_pred = np.argmax(predictions,axis=-1)
    y_pred = np.expand_dims(y_pred,axis=-1)
    
    y_true = np.argmax(ts_lab,axis=-1)
    y_true = np.expand_dims(y_true,axis=-1)
    
    y_true_flat = np.ndarray.flatten(y_true)
    y_pred_flat = np.ndarray.flatten(y_pred)
    ####################################################################################################################
    ## METRICS        
    # CONFUSION MATRIX
    print('\n')
    print('Confusion matrix:')
    cm = metrics.confusion_matrix(y_true, y_pred)
    print(cm)
    
    print('\n')
    tp, fp, tn, fn = perf_measure(cm)
    tp, fp, tn, fn = np.asarray(tp), np.asarray(fp), np.asarray(tn), np.asarray(fn)
    print('True Positive: ', str(tp))
    print('False Positive: ', str(fp))
    print('True Negative: ', str(tn))
    print('False Negative: ', str(fn))
    
    # ACCURACY
    print('\n')
    acc = metrics.accuracy_score(y_true_flat,y_pred_flat)
    accuracies[i] = acc
    print('Total accuracy: ')
    print(acc)
    '''
    #accuraciesClass[i] = cm.diagonal()/cm.sum(axis=1)
    accuraciesClass[i] = (tp+tn)/(tp+tn+fp+fn)
    print('Accuracy per class: ')
    for j in range(len(activities)):
        print('%d'%(j)+'\t'+ str(accuraciesClass[i,j]))
    '''
    # CLASSIFICATION REPORT
    print('\n')
    print('Classification report:')
    reportString = metrics.classification_report(y_true_flat, y_pred_flat, digits=3)
    print(reportString)
    report = metrics.classification_report(y_true_flat, y_pred_flat, digits=3, output_dict=True)
    
    for j in range(len(classes)):
        precision[i,j] = report[classes[j]][measure[0]]
        recall[i,j] = report[classes[j]][measure[1]]
        fScore[i,j] = report[classes[j]][measure[2]]
        support[i,j] = report[classes[j]][measure[3]]
    
    
    # SPECIFITY
    print('\n')
    print('Specifity: ')
    specifity = tn/(tn+fp)
    for j in range(len(activities)):
        print('%d'%(j)+'\t'+ str(specifity[j]))
        specifities[i,j] = specifity[j]
    
    fo = open('./01AUGNORMFFT_SP_KFOLD_models/metrics_'+subject_test+'.txt', "w")
    fo.seek(0,2)
    fo.write('Accuracy: ' + str(acc))
    fo.write('\n')
    fo.write('Confusion matrix: ')
    fo.write('\n')
    fo.write(str(cm))
    fo.write('\n')
    fo.write('Specifity:')
    fo.write(str(specifity))
    fo.write('\n')
    fo.write('Report: ')
    fo.write('\n')
    fo.write(reportString)
    fo.close()
    
    
    del subject_test, ts_seg, ts_lab, model, y_pred, y_true, predictions, y_true_flat, y_pred_flat, reportString, specifity, cm, report
    

##################################################################################################################################
#####################################################  K-FOLD 1  #################################################################
Test subject: 01
Test dataset: 
(2375, 1, 128, 40) (2375, 11)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Confusion matrix:
[[216   0   0   0   0   0   0   0   0   0   0]
 [  0 209   0   7   0   0   0   0   0   0   0]
 [  0   0 216   0   0   0   0   0   0   0   0]
 [  0   0   0 216   0   0   0   0   0   0  

In [9]:
'''
acc = np.sum(accuraciesClass,axis=0)/12
print('Accuracy per class: ')
for j in range(11):
    print('%d'%(j)+'\t'+str(acc[j]))
print('Total accuracy: ' + str(sum(acc)/11))
'''
print('Total accuracy: ' + str(sum(accuracies)/12))

prec = np.sum(precision,axis=0)/12
print('Precision per class: ')
for j in range(11):
    print('%d'%(j)+'\t'+str(prec[j]))
print('Total precision: ' + str(sum(prec)/11))

rec = np.sum(recall,axis=0)/12
print('Recall per class: ')
for j in range(11):
    print('%d'%(j)+'\t'+str(rec[j]))
print('Total recall: ' + str(sum(rec)/11))

fS = np.sum(fScore,axis=0)/12
print('F1-score per class: ')
for j in range(11):
    print('%d'%(j)+'\t'+str(fS[j]))
print('Total F1-score: ' + str(sum(fS)/11))

spec = np.sum(specifities,axis=0)/12
print('Specifities per class: ')
for j in range(11):
    print('%d'%(j)+'\t'+str(spec[j]))
print('Total specifities: ' + str(sum(spec)/11))

Total accuracy: 0.9394736842105263
Precision per class: 
0	0.9802709069493521
1	0.9511648008654686
2	0.9701129761270333
3	0.9817602577486731
4	0.9947812971342384
5	0.9474196770628757
6	0.8786200753137671
7	0.9427960542441438
8	0.9405698033935431
9	0.9305621610347968
10	0.9273317722904348
Total precision: 0.949580889287666
Recall per class: 
0	0.9429012345679012
1	0.9463734567901233
2	1.0
3	0.9035493827160493
4	0.9972993827160493
5	0.9378858024691358
6	0.8279320987654321
7	0.9969135802469135
8	0.958719135802469
9	0.8530092592592592
10	0.9697674418604652
Total recall: 0.9394864341085273
F1-score per class: 
0	0.954170248606207
1	0.9346523358812647
2	0.9838769982906389
3	0.9301836669231193
4	0.9959842823126593
5	0.9304743918001551
6	0.8337195717514009
7	0.9665370188705111
8	0.9409575482594726
9	0.8790034148026411
10	0.9368875956436415
Total F1-score: 0.9351315521037921
Specifities per class: 
0	0.9974139262004015
1	0.9931681333950904
2	0.9964875714065154
3	0.9979542998301683
4	0.999459626